In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("LAB_OPEN_ROUTER_API_KEY")  # must be an or- key

client = OpenAI(
    api_key=API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="meta-llama/llama-3-70b-instruct",
    messages=[
        {"role": "user", "content": "What is 1+3?"}
    ]
)

print(response.choices[0].message.content)


1 + 3 = 4


Patient Risk Sim Prompt:

In [10]:
def patient_risk_simulation_llama(model, prompt, client):
    
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": (
                    "Assign a numerical risk score from 0–10 (10 being the highest risk) " 
                    "for hospitalization for a person with the following characteristics: "
                    f"{prompt}. "
                    "Respond only with:\n"
                    "1. Risk Assessment Score\n"
                    "2. A list of every single parameter and their associated numerical positive or negative risk values "
                    "that add up to the Risk Assessment Score\n"
                    "3. A one to two sentence rationale.\n"
                    "Exclude baseline risk in all calculations."
                )
            }
        ]
    )
    
    return completion.choices[0].message.content


**Llama Pipeline**

In [11]:
import pandas as pd

# Load data and prepare output
# -------------------------
df = pd.read_csv("../synthetic_data1.csv")
num_patients = 50
sim_reps = 5
folder_name = "full_sim_run_1"
os.makedirs(folder_name, exist_ok=True)

results = []

# -------------------------
# Main loop
# -------------------------
for patient_idx in range(num_patients):
    row = df.iloc[patient_idx]
    patient_prompt = ", ".join([f"{col}: {row[col]}" for col in df.columns])

    for sim_idx in range(sim_reps):
        out = {
            "Patient_ID": patient_idx,
            "Simulation_Number": sim_idx
        }

        # ---- LLaMA call ----
        result = patient_risk_simulation_llama(
            model="meta-llama/llama-3-70b-instruct",
            prompt=patient_prompt,
            client=client
        )

        # Save raw output
        file_path = os.path.join(
            folder_name, f"output_patient{patient_idx}_sim{sim_idx}.txt"
        )
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(result)

        # ---- Parsing logic (unchanged) ----
        text = result.replace("–", "-").strip()
        lines = [line.strip() for line in text.splitlines() if line.strip()]

        # 1. Risk score
        score_val = None
        for i, line in enumerate(lines):
            if "Risk Assessment Score" in line:
                for j in range(i + 1, min(i + 4, len(lines))):
                    try:
                        score_val = float(lines[j])
                        break
                    except ValueError:
                        continue
                break
        out["Risk_Assessment_Score"] = score_val

        # 2. Parameter table
        start = None
        for i, line in enumerate(lines):
            if "Parameter" in line and "Value" in line:
                start = i + 1
                break

        if start is not None:
            for line in lines[start:]:
                if not line.startswith("|"):
                    break
                parts = [p.strip() for p in line.split("|") if p.strip()]
                if len(parts) == 2:
                    name, val = parts
                    try:
                        out[name] = float(val)
                    except ValueError:
                        pass

        # 3. Rationale
        rationale = []
        capture = False
        for line in lines:
            if "Rationale" in line:
                capture = True
                continue
            if capture:
                rationale.append(line)
        out["Rationale"] = " ".join(rationale)

        results.append(out)
        print(f"Completed patient {patient_idx}, sim {sim_idx}")

# -------------------------
# Save all results
# -------------------------
pd.DataFrame(results).to_csv("parsed_output_fill_in_nones.csv", index=False)


Completed patient 0, sim 0
Completed patient 0, sim 1
Completed patient 0, sim 2
Completed patient 0, sim 3
Completed patient 0, sim 4
Completed patient 1, sim 0
Completed patient 1, sim 1
Completed patient 1, sim 2
Completed patient 1, sim 3
Completed patient 1, sim 4
Completed patient 2, sim 0
Completed patient 2, sim 1
Completed patient 2, sim 2
Completed patient 2, sim 3
Completed patient 2, sim 4
Completed patient 3, sim 0
Completed patient 3, sim 1
Completed patient 3, sim 2
Completed patient 3, sim 3


KeyboardInterrupt: 